<a href="https://colab.research.google.com/github/Lor3nzoVis3/DeepLearning_ProjectLV_GB/blob/master/FromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
@author: LorenzoVisentini
"""
model = "FromScratch"

#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
#keras
# import keras
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras import optimizers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Images parameters
img_width = 178 
img_height = 218

# train_dir = './source_light/training'
# validation_dir = './source_light/validation' 
train_dir = '/content/drive/My Drive/Deep Learning Project - Image Classification/orient/train'
validation_dir = '/content/drive/My Drive/Deep Learning Project - Image Classification/orient/valid' 

batch_size = 64
epochs = 10

In [ ]:
#model
model = Sequential()

model.add(Conv2D(8), kernel_size = (3,3), activation= 'relu')
model.add(Conv2D(8), kernel_size = (3,3), activation= 'relu')
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides = 1))
model.add(Dropout(0.5))

model.add(Conv2D(8), kernel_size = (5,5), activation= 'relu')
model.add(Conv2D(8), kernel_size = (5,5), activation= 'relu')
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides = 2))
model.add(Dropout(0.5))

model.add(Conv2D(16), kernel_size = (5,5), activation= 'relu')
model.add(Conv2D(16), kernel_size = (5,5), activation= 'relu')
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides = 2))
model.add(Dropout(0.5))

model.add(Conv2D(16), kernel_size = (5,5), activation= 'relu')
model.add(Conv2D(16), kernel_size = (5,5), activation= 'relu')
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides = 2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
model.add(Dense(512, activation= 'relu'))
model.add(BatchNormalization())
model.add(Dense(3), activation= 'softmax')

In [ ]:
model.compile(optimizer = 'Adam',
              loss = 'categorical_crossentropy',  #because we have 3 possible outputs
              metrics = ['accuracy'])

In [ ]:
#preprocessing
datagen = ImageDataGenerator(preprocessing_function = preprocess_input)
train_generator = datagen.flow_from_directory(directory=train_dir,
                                                    target_size=(img_height, img_width),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical",
                                                    shuffle=True,
                                                    seed=42
                                                    )

val_generator = datagen.flow_from_directory(directory=validation_dir,
                                                    target_size=(img_height, img_width),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical",
                                                    shuffle=True,
                                                    seed=42
                                                    )

In [ ]:
history = model.fit_generator(generator = train_generator,
                             steps_per_epoch = train_generator.n//batch_size,
                             epochs = epochs,
                             validation_data = val_generator,
                             validation_steps=val_generator.n//batch_size
                             )

In [ ]:
#save model and history
file_model = '/content/drive/My Drive/Deep Learning Project - Image Classification/Models/'+ 'SourceLight' + modelName + '_' + str(batch_size) + '_' + str(epochs)
model.save(file_model + '.h5')
print(file_model)
#save history
file_hist = '/content/drive/My Drive/Deep Learning Project - Image Classification/History/'+ 'SourceLight' + modelName + '_' + str(batch_size) + '_' + str(epochs) + '.csv'
df_history = pd.DataFrame(history.history)
with open(file_hist, 'w') as file:
    df_history.to_csv(file)

In [ ]:
#plot results
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#evaluation
# https://stackoverflow.com/questions/40564936/keras-what-is-the-difference-between-model-evaluate-generator-and-model-predict
steps = val_generator.n // val_generator.batch_size

accuracy_ontest = model.evaluate_generator(val_generator, steps)
predictions_ontest = model.predict_generator(val_generator, steps)
print(accuracy_ontest)
print(predictions_ontest)